In [1]:
using Laplacians

In [2]:
pwd()

"/Users/serbanstan/git/Laplacians.jl/debug/Solver Testing"

In [3]:
include("../../src/isotonicIPM.jl");

In [4]:
solvers = [augTreeSolver, KMPSDDSolver, samplingSDDSolver, AMGSolver]

4-element Array{Function,1}:
 Laplacians.augTreeSolver    
 Laplacians.KMPSDDSolver     
 Laplacians.samplingSDDSolver
 Laplacians.AMGSolver        

In [5]:
function generateInput(a)
    n = a.n
    
    b = zeros(Float64,n);
    b[1] = 1;
    b[n] = -1;
    f = KMPLapSolver(a, verbose=false, tol=1e-1); # use any other solver
    v = f(b)
    v = v / (v[1] - v[n])

    p = sortperm(v);
    ap = symPermuteCSC(a, p);
    # vp = v[p]
    vp = v[p] + rand(n) / sqrt(n);
    at = triu(ap);
    
    return at,vp
end

generateInput (generic function with 1 method)

In [ ]:
n = 3000

curSolver = []
time = []
edges = []
iters = []
precision = []

for i in 1:100
    a = wtedChimera(n,i)
    a,v = generateInput(a);
    
    for solver in solvers
        try
            mysolver = (H -> solver(H,tol=1e-1,maxits=1000,maxtime=20))
            tic()
            @time x,acc,itercount = isotonicIPMrelEps(a,v,1e-5,mysolver)

            push!(time, toc())
            push!(curSolver, solver)
            push!(edges, nnz(a))
            push!(iters, itercount)
            push!(precision, acc)
        catch
            print("Error for ", i, " and ", solver)
        end
    end
    
    println("Done for ", i)
end

 11.409156 seconds (13.23 M allocations: 2.405 GB, 6.27% gc time)
elapsed time: 11.777641191 seconds
 10.224259 seconds (15.64 M allocations: 3.537 GB, 7.32% gc time)
elapsed time: 10.225922202 seconds
 16.752509 seconds (48.92 M allocations: 5.701 GB, 10.33% gc time)
elapsed time: 16.753447979 seconds
 25.071789 seconds (3.85 M allocations: 865.933 MB, 1.22% gc time)
elapsed time: 25.074894218 seconds
Done for 1
  4.962978 seconds (9.81 M allocations: 1.850 GB, 9.38% gc time)
elapsed time: 4.963826911 seconds
  6.610295 seconds (12.86 M allocations: 3.052 GB, 9.53% gc time)
elapsed time: 6.615754849 seconds
 10.988314 seconds (46.05 M allocations: 5.225 GB, 13.49% gc time)
elapsed time: 10.992078744 seconds
 15.802687 seconds (2.39 M allocations: 618.879 MB, 1.02% gc time)
elapsed time: 15.803603989 seconds
Done for 2
  8.460018 seconds (17.40 M allocations: 3.574 GB, 8.98% gc time)
elapsed time: 8.462265384 seconds
 11.123969 seconds (20.66 M allocations: 5.204 GB, 11.47% gc time)
el